In [5]:
import torch
import torch.nn as nn
from analyzer.data import Dataloader, folder2Vol
from vae.model import vae_3d
import matplotlib.pyplot as plt
from tqdm import tqdm

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Device:  cpu


In [12]:
dl = Dataloader("../datasets/r_em/", "../datasets/r_gt/", chunk_size=(5,4096,4096))
em, gt = dl.load_chunk(vol='both')

dl.volume = folder2Vol("../datasets/r_em/")
dl.volume.shape

em data loaded:  (5, 4096, 4096)
gt data loaded:  (5, 4096, 4096)


(11, 4096, 4096)

In [4]:
model = vae_3d.Conv3dVAE_simple().to(device)
criterion = nn.BCELoss(reduction='sum')

Optimizer:  SGD
